In [ ]:
import scipy.io
from sklearn import svm
from sklearn.svm import SVC
from sklearn.decomposition import KernelPCA
import numpy as np
import math
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
import numpy as geek
import networkx as nx
from scipy.signal import butter, lfilter
from scipy import signal
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Embedding, Reshape
import keras
from random import shuffle
from keras.layers import LSTM, GRU
from keras import regularizers
from keras.layers import Layer
import keras.backend as K
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
import statistics
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam
import time

## define onehot labeling function
def onehot_labels(labels):
    return np.eye(7)[labels]  

def get_one_hot(str_col:list):
    labels = []
    for st in str_col:
        labels.append(list(map(int, st.split(","))))
    return labels

def get_one_hot(str_col:list):
    labels = []
    for st in str_col:
        labels.append(list(map(int, st.split(","))))
    return labels

def normalization(signal):
    mean = statistics.mean(signal)
    std = statistics.stdev(signal)

    signal = [(x - mean) / std for x in signal]
    return signal

def get_X_tensors(df, segment_length=170):
    res = []
    for row in df.itertuples():
        tensors = []
        da_001 = list(row)[1: segment_length+1]
        da_001 = normalization(da_001)
        
        da_3 = list(row)[segment_length+1: 2*segment_length+1]
        da_3 = normalization(da_3)
        
        da_10 = list(row)[2*segment_length+1: 3*segment_length+1]
        da_10 = normalization(da_10)
        
        da3_ops = list(row)[3*segment_length+1: ]
        da3_ops = normalization(da3_ops)

        tensors.append(da_001)
        tensors.append(da_3)
        tensors.append(da_10)
        tensors.append(da3_ops)
        res.append(tensors)
    return res

def load_data(file_name='eyedata.csv'):
    df = pd.read_csv(file_name)
    return get_X_tensors(df.iloc[:, 1:], 170), get_one_hot(df.iloc[:, 0])

X, Y = load_data('/content/drive/MyDrive/eyedata.csv')

labels = []
for label in Y:
    if label not in labels:
        labels.append(label)
        
Y_new = []
for y in Y:
    Y_new.append(labels.index(y))

np.random.seed(42)
X = np.array(X)
Y = np.array(Y_new)
permutation = np.random.permutation(len(X))
X = X[permutation]
Y = Y[permutation]

X_tr, X_val, Y_tr, Y_val = train_test_split(X, Y, test_size=0.1, random_state=10)
X_train, X_test, Y_train, Y_test = train_test_split(X_tr, Y_tr, test_size=0.11, random_state=10)

amplitude = np.max(X_train)/2 - np.min(X_train)/2
X_train_noisy_1 = X_train + np.random.normal(0, 0.001*amplitude, X_train.shape)
X_train_noisy_2 = X_train + np.random.normal(0, 0.004*amplitude, X_train.shape)
X_train_noisy_3 = X_train + np.random.normal(0, 0.008*amplitude, X_train.shape)
X_train_noisy_4 = X_train + np.random.normal(0, 0.012*amplitude, X_train.shape)
    
X_train = np.concatenate((X_train, X_train_noisy_1, X_train_noisy_2, X_train_noisy_3, X_train_noisy_4),0)
Y_train = np.concatenate((Y_train, Y_train, Y_train, Y_train, Y_train),0)

Y_train = onehot_labels(Y_train)
Y_val   = onehot_labels(Y_val)
Y_test  = onehot_labels(Y_test)

num_units = 16
## create our sequential model
regressor = Sequential()
regressor.add(GRU(units=num_units, dropout=0.25, return_sequences=False, recurrent_dropout=0.25))
#regressor.add(Dense(7, activation='softmax'))
#regressor.add(LSTM(num_units, input_shape=(4, 170)))#, return_sequences=True))
#regressor.add(LSTM(num_units//4))#, input_shape=(4, 170)))
regressor.add(Dense(7, activation='softmax'))


#sgd = SGD(lr=0.001, momentum=0.9)
adam = Adam(learning_rate=0.001)
#regressor.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
regressor.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
print(X_train.shape)
print(Y_train.shape)

# define the callback
best_model_path = 'best_GRU_model_'+str(num_units)+'_Adam.h5'
checkpoint = ModelCheckpoint(best_model_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# fit the model with the callback
history4 = regressor.fit(X_train, Y_train,
          batch_size=64,
          epochs=80,
          verbose=1,
          validation_data=(X_val, Y_val),
          callbacks=[checkpoint])
regressor.summary()
# load the best model
regressor.load_weights(best_model_path)

# evaluate the best model
score, accuracy_ = regressor.evaluate(X_test, Y_test, verbose=0)

print('Test loss:', score)
print('Test accuracy:', accuracy_)
     

(6725, 4, 170)
(6725, 7)
Epoch 1/80
103/106 [============================>.] - ETA: 0s - loss: 1.8034 - acc: 0.2904
Epoch 1: val_acc improved from -inf to 0.41071, saving model to best_GRU_model_16_Adam.h5
106/106 [==============================] - 4s 18ms/step - loss: 1.7998 - acc: 0.2923 - val_loss: 1.5239 - val_acc: 0.4107
Epoch 2/80
103/106 [============================>.] - ETA: 0s - loss: 1.3281 - acc: 0.5155
Epoch 2: val_acc improved from 0.41071 to 0.52976, saving model to best_GRU_model_16_Adam.h5
106/106 [==============================] - 2s 15ms/step - loss: 1.3256 - acc: 0.5157 - val_loss: 1.2107 - val_acc: 0.5298
Epoch 3/80
105/106 [============================>.] - ETA: 0s - loss: 1.0212 - acc: 0.6400
Epoch 3: val_acc improved from 0.52976 to 0.61905, saving model to best_GRU_model_16_Adam.h5
106/106 [==============================] - 2s 23ms/step - loss: 1.0212 - acc: 0.6401 - val_loss: 1.0079 - val_acc: 0.6190
Epoch 4/80
106/106 [==============================] - ETA: 0